# Task 1

In [10]:
import requests

In [11]:
import pandas as pd

In [12]:
from bs4 import BeautifulSoup

In [13]:
import time

In [14]:
import re

In [34]:
from datetime import datetime

In [ ]:
import json

In [15]:
keywords = ['python', 'парсинг']

In [ ]:
def get_all_info(url, query):
   
    '''общий результат'''
    table = {}
    
    '''достаем ссылки'''
    urls = []
    params = {
        'search_query': 'python' or 'парсинг'
    }
    res = requests.get(url, params)
    soup = BeautifulSoup(res.text, 'html.parser')
    page1 = soup.find_all(class_='tm-pagination__page')
    lastpage = str(page1[-1])
    str1 = re.findall(r'[0-9]', lastpage)
    
    '''узнаем сколько страниц в поиске с помощью костыля'''
    pages = 0
    if len(str1) == 2:
        pages += int(str1[0])
    elif len(str1) == 4:
        pages += int(str1[0] + str1[1])
    elif len(str1) == 6:
        pages += int(str1[0] + str1[1] + str1[2])
    else:
        print ('Too many lists')
        
    '''достаем все имеющиеся ссылки'''
    for i in range(1, pages):
        params['page'] = i
        time.sleep(0.3)
        news_blocks = soup.find_all(class_='tm-articles-list__item')
        articles_intro = list(map(lambda x: x.find('div', class_='tm-article-body'), news_blocks))
        a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
        for link in a_list:
            if 'https' not in link:
                link = 'https://habr.com' + link
                urls.append(link)
            else:
                urls.append(link)
    
    '''Достаем дату, описание и текст'''
    date = []
    title = []
    text = []
    
    news_blocks = soup.find_all(class_='tm-articles-list__item')
    for item in news_blocks:
        try:
            date.append(item.find_all('span', class_='tm-article-snippet__datetime-published').get_text())
            title.append(item.find_all('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').get_text())
            text.append(item.find_all('div', class_='article-formatted-body article-formatted-body_version-2').get_text())
           
        except:
            pass


    table['date'] = date
    table['title'] = title
    table['url'] = urls
    table_df = pd.DataFrame(table)
    
    return table_df
    


all_ = get_all_info('https://habr.com/ru/all/', 'python' or 'парсинг')
alli = all_[0: 10]
alli

# Task 2

In [57]:
mails = ['xxx@x.ru', 'yyy@y.com']

In [58]:
request = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [59]:
params = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0'
    }

In [60]:
table = pd.DataFrame(columns = {'email','date','site','title'})

In [ ]:
for email in mails:
    res = requests.post(request, json={'emailAddresses': [email]},headers = params)
    info = json.loads(res.text) 
    if info:
        for key in info['breaches'].keys():
            title = extracted_data['breaches'][key]['description']
            date = extracted_data['breaches'][key]['publishDate']
            source = extracted_data['breaches'][key]['site']
            site = extracted_data['breaches'][key]['site']
            data = {'email':email,'date':date,'source':source,'title':title}
            table = table.append(data,ignore_index=True)
table